In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

## using RoBERTa trained on twitter sentiment analysis: 
## https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
classifier(["We are very happy to show you the 🤗 Transformers library.", 
            "Now Playing: JAI - VIP Mix by Finesu -> ***",
            "I kind of like you"])

[{'label': 'Positive', 'score': 0.985775887966156},
 {'label': 'Neutral', 'score': 0.9331138730049133},
 {'label': 'Positive', 'score': 0.667307436466217}]

In [2]:
import pandas as pd 
text_df = pd.read_csv('../data/Fact Tables/EmoteFactTable.csv')

In [3]:
text_df

,ID,Emote,Definition 1,Racism Implication?
0,1,Kappa,Kappa is a Twitch emote that is generally used...,0
1,2,LUL,The acronym LUL stands for “Lame Uncomfortable...,0
2,3,TriHard,The TriHard Twitch emote was created to be use...,1
3,4,PogChamp,The PogChamp emote is used to express surprise...,0
4,5,Pog,The PogChamp emote is used to express surprise...,0
...,...,...,...,...
63,64,3Head,"Used when someone says something cheesy, such ...",0
64,65,YEP,An insincere smile that many people use in sit...,0
65,66,Pepega,It is based on the Pepe the Frog character and...,0
66,67,KKona,The KKona Twitch emote is most commonly used a...,0


In [4]:
twitch_chats = text_df['Definition 1'].tolist()

In [8]:
twitch_chats

['Kappa is a Twitch emote that is generally used to relay sarcasm or as an “eye-roll” response to something the Twitch streamer says or does on screen. Known as a “troll” emote, Kappa is often spammed in chat when someone is sarcastic or checking to see if they have the Golden Kappa.',
 'The acronym LUL stands for “Lame Uncomfortable Laugh.” The LUL Twitch emote is used to express laughter and joy and depicts the late streamer, TotalBuscuit. It is often used on Twitch in the same way that you would use a laugh emote when messaging friends on other platforms.',
 'The TriHard Twitch emote was created to be used when something exciting is happening on the screen or to help generate hype in a channel. It is the image of the streamer Trihex smiling. Use it when you’re hyped or when the streamer does something exciting.',
 'The PogChamp emote is used to express surprise or excitement',
 'The PogChamp emote is used to express surprise or excitement',
 'The PogChamp emote is used to express su

In [5]:
list_polarity = classifier(twitch_chats)

In [8]:
list_polarity_label = []
for result in list_polarity:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")
    list_polarity_label.append(result['label'].upper())

label: Neutral, with score: 0.8278
label: Positive, with score: 0.5579
label: Positive, with score: 0.9341
label: Positive, with score: 0.6078
label: Positive, with score: 0.6078
label: Positive, with score: 0.6078
label: Negative, with score: 0.5644
label: Neutral, with score: 0.8326
label: Neutral, with score: 0.8536
label: Neutral, with score: 0.6606
label: Neutral, with score: 0.7448
label: Positive, with score: 0.5979
label: Negative, with score: 0.669
label: Neutral, with score: 0.594
label: Positive, with score: 0.8056
label: Neutral, with score: 0.5911
label: Positive, with score: 0.9542
label: Negative, with score: 0.6981
label: Neutral, with score: 0.6951
label: Negative, with score: 0.6514
label: Negative, with score: 0.639
label: Positive, with score: 0.6598
label: Negative, with score: 0.6499
label: Neutral, with score: 0.4991
label: Positive, with score: 0.8706
label: Neutral, with score: 0.7582
label: Negative, with score: 0.7176
label: Neutral, with score: 0.6087
label:

In [9]:
text_df['BERT_polarity'] = list_polarity_label

In [10]:
text_df.to_csv('../data/Fact tables/EmoteFactTable_BERT.csv')